# FunUQ for MD
## Sam Reeve and Alejandro Strachan
### Replication of: Reeve, S. T. & Strachan, A. Quantifying uncertainties originating from interatomic potentials in molecular dynamics. (Submitted to Modell. Simul. Mater. Sci. Eng. 2018).
### NVT Morse / Exponential-6 at 1500K and 1 atom (NOT pre-run simulations)

#### This notebook goes through all steps of functional uncertainty quantification FunUQ for interatomic potential in molecular dynamics, matching one case from the paper. The main steps are: 
 * Define folders, simulation system, and models
 * (Run simulations)
 * Calculate functional derivatives
 * Calculate correction for quantities of interest due to changing from one function to another

In [ ]:
import sys, os, numpy as np
# Relative path from notebook to module
sys.path.insert(0, '../../lib/')
sys.path.insert(0, '../../lib/FunUQ/')

In [ ]:
# Import FunUQ module
from FunUQ import *

# Provides access to nanoHUB simulation codes (LAMMPS)
from hublib import use

# Utility functions (Austin Zadoks)
from nH_utils import *

In [ ]:
% use lammps-09Dec14
% matplotlib notebook

In [ ]:
# "True" will run new simulations below
# Change after first usage to only analyze results
run_main = True
run_verify = True
run_perturb = False
run_bruteforce = False

## System setup: define interatomic potentials and quantities of interest

In [ ]:
rundir = os.getcwd()
startdir = os.path.abspath(os.path.join(rundir, 'init/'))

mainname = 'main' # morse
correctname = 'exp6'

In [ ]:
Pot_main = Potential('morse', paramdir=startdir, create=True, N=7000, rmax=7.0, cut=6.0)
Pot_correct = Potential('exp6', paramdir=startdir, create=True, N=7000, rmax=7.0, cut=6.0)

In [ ]:
ax1 = Pot_main.plot()
ax1 = Pot_correct.plot(ax=ax1, color='red')

In [ ]:
QoI_list = ['PotEng', 'Press']
Nqoi = len(QoI_list)
QoI_dict = {'description': 'Replication of Reeve and Strachan, (Submitted 2018)',
            'Ncopies': 2,
            'units': ['eV/atom', 'GPa'],
            #'overwrite': True, 
           }

In [ ]:
QoI = QuantitiesOfInterest(QoI_list, Pot_main,
                           startdir, rundir, mainname, 'metal',
                           input_dict=QoI_dict)
QoI_correct = QuantitiesOfInterest(QoI_list, Pot_correct,
                                   startdir, rundir, correctname, 'metal',
                                   input_dict=QoI_dict)

## Run simulations or extract results

In [ ]:
if run_main:
    QoI.run_lammps(mode='nanoHUB_submit') # 'nanoHUB_local'
if run_verify:
    QoI_correct.run_lammps(mode='nanoHUB_submit')

In [ ]:
submit_status()
#kill_jobs('') # Use RunName
#kill_all_jobs()
local_status(rundir, [mainname, correctname])

In [ ]:
QoI.extract_lammps()
QoI_correct.extract_lammps()
print(QoI); print(QoI_correct)

## Calculate functional derivatives

In [ ]:
FD_dict = {'alist': [-1e-8, -2e-8, 1e-8, 2e-8],
          }

In [ ]:
FuncDer = FuncDer_perturb_coord(QoI, Pot_main,
                                 input_dict=FD_dict)

In [ ]:
if run_bruteforce and FuncDer.method == 'bruteforce':
    FuncDer.run_lammps()
elif run_perturb and FuncDer.method == 'perturbative_allatom':
    FuncDer.rerun_gauss()

In [ ]:
FuncDer.prepare_FD()
FuncDer.calc_FD()
for x in range(Nqoi):
    FuncDer.write_FD(x)
    FuncDer.plot_FD(x)

In [ ]:
FuncDer.plot_perturb(0)

## Correct quantities of interest

In [ ]:
Correct = FunUQ(Pot_main, Pot_correct, QoI.Q_names, QoI.Qavg, QoI_correct.Qavg, 
                Q_units=QoI.units, FD=FuncDer.funcder, R=FuncDer.rlist)

#### Compare this plot to similar case in Reeve & Strachan 2018, Figure 3

In [ ]:
Correct.discrepancy()
Correct.plot_discrep()

In [ ]:
Correct.correct()

#### Compare this plot to similar case in Reeve & Strachan 2018, Figure 3

In [ ]:
for x in range(Nqoi):
    Correct.plot_funcerr(x)
    Correct.plot_correction(x)